In [5]:
import requests
from bs4 import BeautifulSoup
import math
import re
import pandas as pd
from datetime import datetime


In [6]:
base_url = "https://www.postjobfree.com"
URL = "https://www.postjobfree.com/jobs?q=data+engineer&l=Texas&radius=25"
response = requests.get(URL)
content = response.content

In [10]:
soup = BeautifulSoup(content,'html.parser')
# print(soup)

In [11]:
def scrape_page(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content,'html.parser')
    return soup

In [12]:
def scrape_one_page(job_links):
    data_list = []
    for job_list in job_links:
        single_page_scrape = scrape_page(job_list)


        # Find the job title
        title = single_page_scrape.find('h1',id ='titleH1')

        if title:
           job_title = title.text.strip()
        else:
            job_title = single_page_scrape.find("h1", itemprop="title")

            if job_title is not None:
                job_title = job_title.text.strip()
            else:
                job_title = " "


        #Job Url
        job_url = job_list

        #Company Name
        company_name = single_page_scrape.find("span", itemprop='name')
        if company_name is not None:
            company_name = company_name.text.strip()
        else:
            company_name = " "

        #Location
        location = single_page_scrape.find("a", class_='colorLocation')
        if location is not None:
            location = location.text.strip()
        else:
            location = " "

        # Salary
        salary = None
        salary_label = single_page_scrape.find("div", class_="labelHeader", text="Salary:")
        if salary_label is not None:
            salary = salary_label.find_next_sibling(text=True).strip()

        # Posted
        posted = single_page_scrape.find('span',class_='colorDate')
        if posted is not None:
            posted= posted.text.strip()
        else:
            posted=" "

        date_obj = datetime.strptime(posted, "%B %d, %Y")
        formatted_date = date_obj.strftime("%Y-%m-%d")

        # Description
        description_label = single_page_scrape.find("div", class_='normalText')
        description_label=description_label.find_all('p')
        description_text = [p.get_text() for p in description_label]

            
        data = {
            'job_title':job_title,
            'job_url':job_url,
            'company_name':company_name,
            'location': location,
            'salary': salary,
            'posted': formatted_date,
            'description': description_text
        }
        data_list.append(data)
    print(data_list)
    return data_list


In [13]:
# page_no = soup.find('table',class_='Odd normalText')

td_tag = soup.find("td", style="text-align:right;")

if td_tag is not None:
    job_counts = [int(b.text) for b in td_tag.find_all("b")]
    job_count= job_counts[2]
    print(job_count)

page_no = math.ceil((job_count)/10)
page_no


5896


590

In [14]:
job_links = []
i = 1

while i <= 2:
    complete_job_links = []
    print('Page number', i)
    scrape_result = scrape_page(f"https://www.postjobfree.com/jobs?q=data+engineer&l=Texas&radius=25&p={i}")
    h3_tags = scrape_result.find_all("h3", class_="itemTitle")
    for h3_tag in h3_tags:
        a_tag = h3_tag.find("a")
        if a_tag is not None:
            href = a_tag.get("href")
            # print(href)

            job_link = base_url+href
            job_links.append(job_link)
            
    i = i + 1
print(job_links)
# len(job_links)


Page number 1
Page number 2
['https://www.postjobfree.com/job/r99chd/controls-engineer-odessa-tx', 'https://www.postjobfree.com/job/sa6u6s/materials-engineer-san-antonio-tx', 'https://www.postjobfree.com/job/sbhwf4/mechanical-engineer-quintana-tx', 'https://www.postjobfree.com/job/sazo1o/mechanical-machine-design-san-antonio-tx', 'https://www.postjobfree.com/job/sezyc2/data-engineer-temple-tx', 'https://www.postjobfree.com/job/se0jag/data-engineer-irving-tx-75039', 'https://www.postjobfree.com/job/q7uf05/sr-data-engineer-etl-houston-tx-77002', 'https://www.postjobfree.com/job/sc8bey/data-engineer-iii-senior-dallas-tx-75201', 'https://www.postjobfree.com/job/se0dny/field-service-engineer-pasadena-tx-77505', 'https://www.postjobfree.com/job/sezbmv/field-service-engineer-pasadena-tx-77505', 'https://www.postjobfree.com/job/se4kqt/data-engineer-pchp-dallas-tx-75201', 'https://www.postjobfree.com/job/se121z/data-engineer-houston-tx-77024', 'https://www.postjobfree.com/job/sb49xc/data-engine

In [15]:

page_data = scrape_one_page(job_links)


/tmp/ipykernel_226277/1193634000.py:40: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  salary_label = single_page_scrape.find("div", class_="labelHeader", text="Salary:")
/tmp/ipykernel_226277/1193634000.py:42: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  salary = salary_label.find_next_sibling(text=True).strip()


[{'job_title': 'Controls Engineer', 'job_url': 'https://www.postjobfree.com/job/r99chd/controls-engineer-odessa-tx', 'company_name': 'Retained Talent Acquisition Group', 'location': 'Odessa, TX', 'salary': '130,000 - 160,000', 'posted': '2023-06-26', 'description': ['6+ years of Control Engineering experience with designing control systems', 'Develop control system deliverables for all phases of a project including conceptual design, preliminary design, and detailed', 'design.', 'Develop the design of PLC based control systems.', 'Develop cause & effects matrix and control narratives.', 'Develop typical control system engineering and design drawings such as: PLC, RIO, & J-Box schematics; enclosure layout drawings; network drawing; area classification plans; control equipment arrangement plans.', 'Develop control system/instrumentation specifications.', 'Develop the design of miscellaneous electrical systems such as lighting, grounding, electrical heat tracing, etc.', 'Develop typical c

In [16]:
df =pd.DataFrame(page_data)

In [17]:
df.to_csv('postjobfree.csv',index=False)
df


,job_title,job_url,company_name,location,salary,posted,description
0,Controls Engineer,https://www.postjobfree.com/job/r99chd/control...,Retained Talent Acquisition Group,"Odessa, TX","130,000 - 160,000",2023-06-26,[6+ years of Control Engineering experience wi...
1,Materials Engineer,https://www.postjobfree.com/job/sa6u6s/materia...,Top Talent Central,"San Antonio, TX",None,2023-06-30,[**EDUCATION: MINIMUM MASTER'S DEGREE WITH PHD...
2,Mechanical Engineer,https://www.postjobfree.com/job/sbhwf4/mechani...,Top Talent Central,"Quintana, TX",None,2023-07-01,"[6am - 4pm or 8am-6pm (Schedule is flexible), ..."
3,Mechanical Machine Design Engineer II,https://www.postjobfree.com/job/sazo1o/mechani...,Top Talent Central,"San Antonio, TX",None,2023-07-01,[***EDUCATION REQUIRED - BACHELOR'S DEGREE IN ...
4,Data Engineer,https://www.postjobfree.com/job/sezyc2/data-en...,McLane,"Temple, TX",None,2023-07-17,"[JOB SUMMARY / GENERAL DESCRIPTION:, Cleans, p..."
5,Data Engineer,https://www.postjobfree.com/job/se0jag/data-en...,CVS Health,"Irving, TX, 75039",None,2023-07-17,"[Requisition Number, 2544215BR, , Position Sum..."
6,Sr. Data Engineer (ETL / Java / Big Data),https://www.postjobfree.com/job/q7uf05/sr-data...,Helmsman Group,"Houston, TX, 77002","$175,000.00",2023-04-17,"[Sr. Data Engineer (ETL / Java / Big Data), , ..."
7,Data Engineer: III (Senior),https://www.postjobfree.com/job/sc8bey/data-en...,Robert Half,"Dallas, TX, 75201",None,2023-07-08,[Ref ID: 04380-0012725502 Classification: Data...
8,Field Service Engineer II,https://www.postjobfree.com/job/se0dny/field-s...,Honeywell,"Pasadena, TX, 77505",None,2023-07-17,[Join the industry leader to design the next g...
9,Field Service Engineer II,https://www.postjobfree.com/job/sezbmv/field-s...,Honeywell,"Pasadena, TX, 77505",None,2023-07-17,[Join the industry leader to design the next g...
